In [4]:
# sourcery skip: avoid-global-variables

import pandas as pd
import os

import skombo
from tabulate import tabulate
import matplotlib
from pandas.io.formats.style import Styler
from skombo.fd_ops import get_fd,clean_fd,FdBotCsvManager,FrameData

get_fd()
FD = clean_fd()

LOG = skombo.LOG


test_combo_csv_path = os.path.join(
    skombo.ABS_PATH,
    (skombo.CHARS.AN.lower() + skombo.TEST_COMBOS_SUFFIX),
)
pd.options.display.max_rows = None # type: ignore
pd.options.display.max_columns = None # type: ignore
pd.options.display.width = None # type: ignore
pd.options.display.max_colwidth = None # type: ignore
pd.options.display.precision = 2 # type: ignore
pd.options.display.float_format = "{:,.2f}".format # type: ignore
matplotlib.use("nbagg") # type: ignore

In [5]:
from skombo import COLS
from skombo import CHARS
from skombo.utils import filter_dict
#FD.loc[CHARS.AN, columns_to_keep].style.pipe(make_look_good)  # type: ignore

def make_look_good(styler: Styler):
    styler.set_caption("Frame Data")
    styler.background_gradient(cmap="Blues")
    return styler


columns_to_exclude = [
    COLS.footer,
    COLS.thumb_url,
    COLS.footer_url,
    COLS.char,
    COLS.m_name,
]
columns_to_keep = dict(COLS.__dict__)
columns_to_keep = list(
    filter_dict(columns_to_keep, columns_to_exclude, filter_values=True).values()
)



In [6]:
# find any columns with any list containing an int
# all rows and indexes
# check all columns to see if any values are a list
from skombo.utils import timer_func


def contains_numeric_lists(data, column):
    """
    Returns True if the specified column of the given data contains any list
    containing at least one string that represents an integer.
    """

    for value in data[column]:
        if isinstance(value, list):
            for item in value:
                if (
                    isinstance(item, str) and item.strip("-").isnumeric()
                ) or isinstance(item, (int, float)):
                    return True

    return False


@timer_func
def filter_test():
    numeric_cols = [col for col in FD.columns if contains_numeric_lists(FD, col)]
    filtered = FD[numeric_cols] if numeric_cols else None


filter_test()
